In [12]:
# destytojo pvz

import PySimpleGUI as sg

layout = [
    [sg.Text("Koks tavo vardas?", font="Verdana 15")],
    [sg.Input(key="-NAME-", font="Terminal 15")],
    [
        sg.Button("Pasisveikinti", key="-HELLO-"), 
        sg.Button("Atsisveikinti", key="-BYE-"),
    ],
    [sg.Text(size=(40, 1), key="-OUTPUT-", font="Verdana 15")],
]

window = sg.Window("LABAS", layout)
sg.DEFAULT_FONT

while True:
    event, values = window.read()
    if event == sg.WINDOW_CLOSED:
        break

    if event == "-HELLO-":
        window["-OUTPUT-"].update(
            f"Sveiki {values['-NAME-']}",
            text_color="#99ff99"
        )
    elif event == "-BYE-":
        window["-OUTPUT-"].update(
            f"Viso gero {values['-NAME-']}",
            text_color="#ff9999"
        )

window.close()

In [4]:
# mano - veikia!

import PySimpleGUI as sg

layout = [
    [sg.Text("Koks jusu vardas, ar galite paskolinti pinigu?")],
    [sg.Input(key="-VARDAS-")],
    [
        sg.Button("Negaliu!", key="-ATE-"),
        sg.Button("Galiu!", key="-VALIO-")
    ]
]

layout1 = [
    [sg.Text("Kiek?")],
    [sg.Input(key="-SUMA-")],
    [
        sg.Button("Skolinu!", key="-EXIT-"),
    ]
]

window = sg.Window("Labas", layout)
window1 = sg.Window("Suma €", layout1)

while True:
    event, values = window.read()
    if event == "-ATE-":
        break
    event, values = window1.read()
    if event == "-EXIT-":
        break



window.close()
window1.close()